In [33]:
import sys
# Aggiungi il percorso della cartella che contiene utils.py a sys.path
sys.path.append('../utils')

In [34]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
from sklearn.multioutput import MultiOutputRegressor
sns.set_theme(style='darkgrid')

from sklearn.model_selection import GridSearchCV, learning_curve

from sklearn.metrics import make_scorer
from utils import save_plot, mean_euclidean_error, root_mean_squared_error, multidim_r2
from joblib import dump

In [35]:
TRAIN_DATA = os.path.join('..', '..', 'datasets', 'cup2023', 'ML-CUP23-TR.csv')
IMAGES_FOLDER = os.path.join('..', '..', 'images', 'cup2023', 'SVR')
MODEL_FOLDER = os.path.join('..', '..', 'trained_models', 'cup2023')

In [36]:
loadGridSearch = True
loadGridSearchPath = os.path.join(MODEL_FOLDER,'GridSearchLog', 'grid_search_SVR_results.csv')

In [37]:
df_grid = pd.read_csv(loadGridSearchPath, sep=',')

In [38]:
df_grid.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rf__estimator__C,param_rf__estimator__epsilon,param_rf__estimator__gamma,param_rf__estimator__kernel,params,split0_test_MEE,...,std_test_R2,rank_test_R2,split0_test_RMSE,split1_test_RMSE,split2_test_RMSE,split3_test_RMSE,split4_test_RMSE,mean_test_RMSE,std_test_RMSE,rank_test_RMSE
0,0.125174,0.027501,0.042007,0.011753,0.1,0.01,0.001,rbf,"{'rf__estimator__C': 0.1, 'rf__estimator__epsi...",-35.999280,...,0.029072,135,-23.825033,-25.604234,-27.101993,-26.076928,-24.507458,-25.423129,1.155093,135
1,0.126812,0.030435,0.008041,0.002278,0.1,0.01,0.001,linear,"{'rf__estimator__C': 0.1, 'rf__estimator__epsi...",-10.610135,...,0.003983,84,-7.907564,-8.615893,-8.624024,-8.402639,-7.918123,-8.293649,0.320920,84
2,0.113448,0.031491,0.014008,0.006021,0.1,0.01,0.001,poly,"{'rf__estimator__C': 0.1, 'rf__estimator__epsi...",-36.291608,...,0.034048,144,-24.047223,-25.820120,-27.449322,-26.379864,-24.771241,-25.693554,1.194222,144
3,0.110361,0.021443,0.037942,0.023710,0.1,0.01,0.010,rbf,"{'rf__estimator__C': 0.1, 'rf__estimator__epsi...",-33.977642,...,0.015622,123,-22.479938,-24.086807,-25.180768,-24.382349,-23.051082,-23.836189,0.961749,122
4,0.058126,0.021068,0.003539,0.001092,0.1,0.01,0.010,linear,"{'rf__estimator__C': 0.1, 'rf__estimator__epsi...",-10.610135,...,0.003983,84,-7.907564,-8.615893,-8.624024,-8.402639,-7.918123,-8.293649,0.320920,84


In [39]:
df_grid.head()
best_params = df_grid.loc[df_grid['rank_test_MEE'] == 1, 'params'].values[0]

In [40]:
best_params # {'C': 100, 'epsilon': 0.1, 'gamma': 0.01, 'kernel': 'rbf'} TODO : da qui si puo vedere che probabilmente, il parametro C serve piu alto

"{'rf__estimator__C': 100, 'rf__estimator__epsilon': 0.1, 'rf__estimator__gamma': 0.1, 'rf__estimator__kernel': 'rbf'}"

In [41]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 14))

df_train = pd.read_csv(TRAIN_DATA, header=None, comment='#', usecols=columns_to_read, delimiter=',')
df_train = df_train.astype('float64') #casting
df_train.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [42]:
features = ['feature_' + str(i) for i in range(1, 11)]
targets = ['target_x', 'target_y', 'target_z']

# Rename columns
new_column_names = features + targets
df_train.columns = new_column_names

df_train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,target_x,target_y,target_z
0,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [43]:
from sklearn.model_selection import train_test_split

X = df_train[features].to_numpy()
y = df_train[targets].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #TODO split potenzialmente variabile?

# Create model

In [44]:
mee_scorer = make_scorer(mean_euclidean_error, greater_is_better=False)
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)
multidim_r2_scorer = make_scorer(multidim_r2, greater_is_better=True)

scoring = {'MEE': mee_scorer, 'R2': multidim_r2_scorer, 'RMSE': rmse_scorer}

In [45]:
# Definizione del modello SVR
svr = svm.SVR()

# Creazione di un MultiOutputRegressor
model = MultiOutputRegressor(svr)

In [46]:
#param_grid = {
 #   'svr__estimator__C': [0.1, 1, 10, 100],
 #   'svr__estimator__gamma': [ 0.001, 0.01, 0.1, 1],
  #  'svr__estimator__kernel' : ['rbf', 'linear', 'poly'],
   # 'svr__estimator__epsilon': [0.01, 0.1, 1]
#}



param_grid = [
    {'svr__estimator__kernel': ['linear'], 'svr__estimator__C': [0.1, 1, 10, 100], 'svr__estimator__epsilon': [0.01, 0.1, 1]},
    {'svr__estimator__kernel': ['rbf'], 'svr__estimator__C': [0.1, 1, 10, 100], 'svr__estimator__gamma': [0.001, 0.0001], 'svr__estimator__epsilon': [0.01, 0.1, 1]},
    {'svr__estimator__kernel': ['poly'], 'svr__estimator__C': [0.1, 1, 10, 100], 'svr__estimator__degree': [2, 3, 4], 'svr__estimator__gamma': [0.001, 0.0001], 'svr__estimator__coef0': [0, 1, 2], 'svr__estimator__epsilon': [0.01, 0.1, 1]},
    {'svr__estimator__kernel': ['sigmoid'], 'svr__estimator__C': [0.1, 1, 10, 100], 'svr__estimator__gamma': [0.001, 0.0001], 'svr__estimator__coef0': [0, 1, 2], 'svr__estimator__epsilon': [0.01, 0.1, 1]}
]

In [47]:
# Definizione del pipeline con RobustScaler e un modello Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
"""
Scales the input data using RobustScaler.

in particular it uses the formula:
    (X - X.median) / IQR
where IQR is the interquartile range (75th - 25th percentile)

we use the RobustScaler because it is robust to outliers. (it uses the median instead of the mean) TODO: do we have outliers?
futhermore, it allows us to have una media di 0 e una varianza di 1, which is good for the neural network
futhermor, it allows us to have a more rapid convergence of the gradient descent, and in general in the optimization algorithms
inoltre ci permette di migliorare gli algo basati sulla distanza (es. KNN, clustering ierarchico), algoritmi che assumono normalità dei dati (alcuni metodi statistici), e algoritmi basati su gradient descent (reti neurali, regressione lineare) beneficiano particolarmente dello scaling.
Non dovrebbero beneficiare particolarmente algo come il Random Forest, che non si basano su distanza, e non assumono normalità dei dati.
Parameters:
X_train (array-like): The input data to be scaled.

Returns:
scaled_data (array-like): The scaled data.

"""

pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('svr', model)
])

In [48]:


grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    refit='MEE',
    n_jobs=-1,
    verbose=2
)
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END svr__estimator__C=0.1, svr__estimator__epsilon=0.01, svr__estimator__kernel=linear; total time=   0.1s
[CV] END svr__estimator__C=0.1, svr__estimator__epsilon=0.1, svr__estimator__kernel=linear; total time=   0.1s
[CV] END svr__estimator__C=0.1, svr__estimator__epsilon=0.01, svr__estimator__kernel=linear; total time=   0.1s
[CV] END svr__estimator__C=0.1, svr__estimator__epsilon=0.01, svr__estimator__kernel=linear; total time=   0.1s
[CV] END svr__estimator__C=0.1, svr__estimator__epsilon=0.01, svr__estimator__kernel=linear; total time=   0.1s
[CV] END svr__estimator__C=0.1, svr__estimator__epsilon=0.01, svr__estimator__kernel=linear; total time=   0.1s
[CV] END svr__estimator__C=0.1, svr__estimator__epsilon=0.1, svr__estimator__kernel=linear; total time=   0.1s
[CV] END svr__estimator__C=0.1, svr__estimator__epsilon=0.1, svr__estimator__kernel=linear; total time=   0.0s
[CV] END svr__estimator__C=0.1, svr__estima

KeyboardInterrupt: 

In [ ]:
results = pd.DataFrame(grid_search.cv_results_)
results.to_csv('grid_search_SVR_results.csv', index=False)


In [ ]:
'''
Quando si utilizza grid_search.best_score_ in scikit-learn e si ottiene un valore negativo,
ciò è tipicamente dovuto al fatto che scikit-learn trasforma alcune funzioni di scoring in modo 
che possano essere trattate come funzioni di "massimizzazione". '''


best_params = grid_search.best_params_
best_index = grid_search.best_index_

mee = grid_search.best_score_
r2 = grid_search.cv_results_['mean_test_R2'][best_index]
rmse = grid_search.cv_results_['mean_test_RMSE'][best_index]

print('Best parameters:', best_params)
print('Best MEE score:', abs(mee))
print('R2:', abs(r2))
print('RMSE:', abs(rmse))

In [ ]:
adapted_params = {'C': best_params['svr__estimator__C'],
                  'gamma': best_params['svr__estimator__gamma'],
                  'kernel': best_params['svr__estimator__kernel'],
                  'epsilon': best_params['svr__estimator__epsilon']}

In [ ]:
# Definizione del modello SVR
svr = svm.SVR(**adapted_params)

# Creazione di un MultiOutputRegressor
model = MultiOutputRegressor(svr)

final_model = Pipeline([
    ('scaler', RobustScaler()),
    ('svr', model)
])

final_model.fit(X_train, y_train)

# Learning curve

In [ ]:
train_sizes, train_scores, validation_scores = learning_curve(
    estimator=final_model,
    X=X_train,
    y=y_train,
    train_sizes=[0.1, 0.33, 0.55, 0.78, 1.],
    cv=5,
    scoring=mee_scorer
)

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
validation_scores_mean = np.mean(validation_scores, axis=1)
validation_scores_std = np.std(validation_scores, axis=1)

In [ ]:
plt.figure()
plt.title('Learning curve', fontweight='bold', fontsize=16)
plt.xlabel('Training examples', fontweight='bold')
plt.ylabel('Absolute Error', fontweight='bold')
plt.grid(True)

color1 = sns.dark_palette((20, 60, 50), input='husl')[-1]
color2 = sns.dark_palette('seagreen')[-1]

# Filling the area around the mean scores to indicate variability of the model's performance
# The shaded area represents the range of scores (mean ± standard deviation) for each training set size
plt.fill_between(
    train_sizes, np.abs(train_scores_mean - train_scores_std),
                 np.abs(train_scores_mean + train_scores_std), alpha=0.2, color=color1
)
plt.fill_between(
    train_sizes, np.abs(validation_scores_mean - validation_scores_std),
                 np.abs(validation_scores_mean + validation_scores_std), alpha=0.2, color=color2
)

# Mean score lines for training and validation
sns.lineplot(x=train_sizes, y=np.abs(train_scores_mean), marker='s', color=color1, label='Training score')
sns.lineplot(x=train_sizes, y=np.abs(validation_scores_mean), marker='s', color=color2, label='Cross-validation score')
plt.legend(loc='lower right')

save_plot(plt, IMAGES_FOLDER, 'learning_curve')
plt.show()

# Plot temp map

In [ ]:
results

In [ ]:
# Define the range of epsilon, gamma, and C
epsilon_range = [0.01, 0.1, 1]
kernel_range = ['rbf', 'linear', 'poly']
gamma_range = [0.001, 0.01, 0.1, 1]
C_range = [0.1, 1, 10, 100]

# Create a subplot grid
fig, axes = plt.subplots(len(epsilon_range), len(kernel_range), figsize=(20, 15))  # Adjust the figsize parameter

# Iterate over epsilon and kernel
for idx, epsilon in enumerate(epsilon_range):
    for kdx, kernel in enumerate(kernel_range):
        # Filter the results for epsilon and kernel
        mask = (results['param_rf__estimator__epsilon'] == epsilon) & (results['param_rf__estimator__kernel'] == kernel)
        scores = np.abs(results.loc[mask, 'mean_test_MEE'].values.reshape(len(C_range), len(gamma_range)))

        # Create the heat map
        ax = axes[idx, kdx]
        cax = ax.imshow(scores, interpolation='nearest', cmap=plt.cm.hot, extent=[0.001, 1, 0.1, 100], vmin=np.min(scores), vmax=np.max(scores), aspect='auto')
        ax.set_title(f'epsilon = {epsilon}, kernel = {kernel}')
        ax.set_xlabel('gamma')
        ax.set_ylabel('C')
        plt.colorbar(cax, ax=ax)
        ax.set_xticks(np.arange(len(gamma_range)))
        ax.set_xticklabels(gamma_range, rotation=45)
        ax.set_yticks(np.arange(len(C_range)))
        ax.set_yticklabels(C_range)
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_ylim([0.1, 100])  # Set the y-axis limits to amplify the scale

# Add axis labels for the outer heat maps
for ax in axes[:, 0]:
    ax.set_ylabel('C')

for ax in axes[-1, :]:
    ax.set_xlabel('gamma')

plt.tight_layout()
plt.show()

# Test of the model

In [ ]:
y_pred = final_model.predict(X_test)

mee = mean_euclidean_error(y_test, y_pred)
r2 = multidim_r2(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)

print('MEE:', mee)
print('R2:', r2)
print('RMSE:', rmse)

# Save model

In [ ]:
model_path = os.path.join(MODEL_FOLDER, 'SVR_model.joblib')
dump(model, model_path, compress=3)